In [1]:
# auto update python file
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
# load sys path
# set path
import sys
sys.path.append('../tracking_validation')

In [4]:
import argparse
import logging
import tempfile
from typing import List
from typing import Optional
from typing import Tuple
from typing import Union
from utils import *

In [5]:
# sample rosbag loading
from rosbag_parser import get_topics_and_tf
#from tracking_parser import DetctionAndTrackingParser

rosbag_file = '/home/yoshiri/Downloads/temp/testcase1/scenario_0_0.db3'
topic_names = ["/perception/object_recognition/detection/objects", "/perception/object_recognition/tracking/objects"]
topics, tf = get_topics_and_tf(rosbag_file, topic_names)

[INFO 1685032682.692980922] [rosbag2_storage]: Opened database '/home/yoshiri/Downloads/temp/testcase1/scenario_0_0.db3' for READ_ONLY. (open() at ./src/rosbag2_storage_default_plugins/sqlite/sqlite_storage.cpp:204)


# Evaluation Test

## Step1: CheckModel

In [6]:
# create sample objects for testing and evaluation
sample_tracked_objects = topics[topic_names[1]][0]
sample_detected_objects = topics[topic_names[0]][0]

import pickle
sample_data = {'tracked_objects': sample_tracked_objects, 'detected_objects': sample_detected_objects}
with open('../test/test_data/sample_objects', 'wb') as f:
    pickle.dump(sample_data,f)    

In [6]:
# load sample objects
import pickle
def load_sample_objects():
    with open('../test/test_data/sample_objects', 'rb') as f:
        sample_data = pickle.load(f)
    sample_tracked_objects = sample_data['tracked_objects']
    sample_detected_objects = sample_data['detected_objects']
    return sample_tracked_objects, sample_detected_objects

sample_tracked_objects, sample_detected_objects = load_sample_objects()

In [7]:
from evaluation_utils import list_dynamic_object_from_ros_msg
import driving_log_replayer.perception_eval_conversions as eval_conversions
from perception_eval.common.status import FrameID
from autoware_auto_perception_msgs.msg import TrackedObject, TrackedObjects, DetectedObject, DetectedObjects, PredictedObject, PredictedObjects
from perception_eval.common.object import DynamicObject
from perception_eval.common.dataset import FrameGroundTruth

In [36]:
# tracked obj to dynamic obj
def tracked_obj_to_dynamic_obj(tracked_obj: TrackedObjects) -> List[DynamicObject]:
    unix_time: int = eval_conversions.unix_time_from_ros_msg(tracked_obj.header)
    frame_id: FrameID = FrameID.from_value(tracked_obj.header.frame_id)
    return list_dynamic_object_from_ros_msg(frame_id, unix_time, tracked_obj.objects)

msg = sample_tracked_objects[1]
tracked_obj_to_dynamic_obj(msg)

In [38]:
# tracked obj to ground truth frame
def tracked_obj_to_frame_ground_truth(tracked_obj: TrackedObjects, frame_name: str) -> FrameGroundTruth:
    unix_time: int = eval_conversions.unix_time_from_ros_msg(tracked_obj.header)
    dynamic_objects: List[DynamicObject] = tracked_obj_to_dynamic_obj(tracked_obj)
    return FrameGroundTruth(unix_time, frame_name, dynamic_objects)

msg = sample_tracked_objects[1]
tracked_obj_to_frame_ground_truth(msg, '1')

In [11]:
sample_tracked_objects[1].objects[0].kinematics, sample_detected_objects[1].objects[0].kinematics

(autoware_auto_perception_msgs.msg.TrackedObjectKinematics(pose_with_covariance=geometry_msgs.msg.PoseWithCovariance(pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=3759.5928970352898, y=73739.71130046145, z=20.556364059448242), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.9713731197630094, w=0.2375589657366735)), covariance=array([0.2097757 , 0.09576888, 0.        , 0.        , 0.        ,
        0.        , 0.09576888, 0.07551026, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.01      ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.01      , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.01      , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.022302  ])), orientation_availability=1, twist_with_covariance=geometry_msgs.msg.TwistWithCovariance(twist=geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Ve

# Prediction evals

In [48]:
from rosbag_parser import get_topics_as_dict

prediction_topic = "/perception/object_recognition/objects"
pred_topic_dicts = get_topics_as_dict(rosbag_file, [prediction_topic])

sample_predicted_objects = pred_topic_dicts[prediction_topic][0]

[INFO 1685294132.139797531] [rosbag2_storage]: Opened database '/home/yoshiri/Downloads/temp/testcase1/scenario_0_0.db3' for READ_ONLY. (open() at ./src/rosbag2_storage_default_plugins/sqlite/sqlite_storage.cpp:204)


In [55]:
predicted_obj = sample_predicted_objects[1].objects[0]

In [59]:
predicted_obj.kinematics.predicted_paths[0].path

[geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=3759.5928970352898, y=73739.71130046145, z=20.556364059448242), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.9713731197630094, w=0.2375589657366735)),
 geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=3759.5928970352898, y=73739.71130046145, z=20.556364059448242), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.9713731197630094, w=0.2375589657366735)),
 geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=3759.5928970352898, y=73739.71130046145, z=20.556364059448242), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.9713731197630094, w=0.2375589657366735)),
 geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=3759.5928970352898, y=73739.71130046145, z=20.556364059448242), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.9713731197630094, w=0.2375589657366735)),
 geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=3759.5928970352898, y=73739.71130046145, 